In [1]:

from cassandra.cluster import Cluster

cluster = Cluster(['cassandra-container'])  # ou use o IP se necessário
session = cluster.connect()


In [2]:
#checando keyspaces
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces")
for row in rows:
    print(row.keyspace_name)

system_auth
system_schema
system_distributed
system
system_traces


In [3]:

import time

# Criar o keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS aulademo
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

# Aguarde alguns segundos para garantir que o keyspace esteja disponível
time.sleep(5)

# Agora defina o keyspace
session.set_keyspace('aulademo')


In [4]:

session.execute("""
    CREATE TABLE IF NOT EXISTS Estudantes (
        id UUID PRIMARY KEY,
        nome TEXT,
        idade INT,
        curso TEXT,
        email TEXT
    )
""")


In [5]:
import uuid

estudantes = [
    (uuid.uuid4(), "Ana", 22, "Engenharia", "ana@email.com"),
    (uuid.uuid4(), "Bruno", 24, "Direito", "bruno@email.com"),
    (uuid.uuid4(), "Carla", 21, "Engenharia", "carla@email.com"),
    (uuid.uuid4(), "Daniel", 23, "Medicina", "daniel@email.com"),
    (uuid.uuid4(), "Eduarda", 25, "Direito", "eduarda@email.com")
]

for est in estudantes:
    session.execute("""
        INSERT INTO Estudantes (id, nome, idade, curso, email)
        VALUES (%s, %s, %s, %s, %s)
    """, est)

In [6]:
def buscar_id_por_nome(nome):
    rows = session.execute("SELECT id FROM Estudantes WHERE nome=%s ALLOW FILTERING", [nome])
    for row in rows:
        return row.id
    return None

# Atualizar curso de Ana
id_ana = buscar_id_por_nome("Ana")
if id_ana:
    session.execute("UPDATE Estudantes SET curso=%s WHERE id=%s", ["Arquitetura", id_ana])

# Excluir Daniel
id_daniel = buscar_id_por_nome("Daniel")
if id_daniel:
    session.execute("DELETE FROM Estudantes WHERE id=%s", [id_daniel])

In [7]:
rows = session.execute("SELECT * FROM Estudantes")
for row in rows:
    print(row)

Row(id=UUID('a698e294-f950-4788-8d03-496dad1972e4'), curso='Direito', email='bruno@email.com', idade=24, nome='Bruno')
Row(id=UUID('6537a386-8409-4268-82b9-60a77c2ce423'), curso='Direito', email='eduarda@email.com', idade=25, nome='Eduarda')
Row(id=UUID('9686d48e-96ae-4b11-b3a4-0f265cc6e139'), curso='Arquitetura', email='ana@email.com', idade=22, nome='Ana')
Row(id=UUID('22b4ced1-db2b-4eee-aff7-ee188dbabdf0'), curso='Engenharia', email='carla@email.com', idade=21, nome='Carla')
